In [1]:
import pandas as pd 
import tensorflow as tf
import numpy as np
import scipy.misc


#from PIL import Image
import PIL.Image as Image
import torchvision.transforms as transforms

to_tensor = transforms.ToTensor()
import torchvision
import torch.nn as nn

import torch.nn.functional as F

from sklearn.model_selection import train_test_split

from functools import partial
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

!pip install torchinfo
from torchinfo import summary

from keras.layers import Dense, GlobalAveragePooling2D
#from  import keras.layers.Add


import tensorflow

import os


from zipfile import ZipFile


2023-06-06 20:23:07.085353: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-06 20:23:07.149165: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-06 20:23:07.150154: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-06 20:23:08.821623: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Defaulting to user installation because normal site-packages is not writeable


In [2]:
biomarker_completeList = ['CD11c','CD14',
 'CD163',
 'CD20',
 'CD3',
 'CD31',
 'CD38',
 'CD4',
 'CD45',
 'CD45RB',
 'CD45RO',
 'CD56',
 'CD57',
 'CD68',
 'CD69',
 'CD8',
 'CK17',
 'ChyTr',
 'Collagen1',
 'ECAD',
 'FAP',
 'FOXP3',
 'Fibronectin',
 'GLUT1',
 'H3K27me3',
 'H3K9ac',
 'HLA1',
 'HLADR',
 'IDO',
 'Ki67',
 'LAG3',
 'PD1',
 'PDL1',
 'SMA',
 'TBET',
 'TCF1',
 'TIM3',
 'Vim']


Some of the biomarkers are functional biomarkers which are not very informative on the cell type. So, we decided to exclude them from the model. We ended up 20 biomarkers listed below. 

In [ ]:
# markerNum = len(biomarkerList)
# biomarker_toKeep = pd.read_excel('Markers to Use.xlsx')
# values = biomarker_toKeep['Marker'].values.tolist()

biomarker_toKeep = ['Calprotectin',
 'CD11c',
 'CD14',
 'CD163',
 'CD20',
 'CD3',
 'CD31',
 'CD4',
 'CD45',
 'CD56',
 'CD68',
 'CD8',
 'ChyTr',
 'CK17',
 'FAP',
 'FOXP3',
 'HLADR',
 'PD1',
 'PDL1',
 'SMA']

### Metadata have the patient IDs, tumor location of the corresponding images, and response to therapy.

In [3]:

metadata = pd.read_excel('/oak/stanford/groups/ccurtis2/users/bgsimon/CS230_CourseProject/simplified_metadata.xlsx')
metadata

,fov,Patient_ID,Localization,iRECIST_response
0,TONIC_TMA2_R4C1,2,Other,non-responders
1,TONIC_TMA2_R4C2,2,Other,non-responders
2,TONIC_TMA2_R4C3,2,Other,non-responders
3,TONIC_TMA2_R9C6,5,Skin,responders
4,TONIC_TMA2_R10C1,5,Skin,responders
...,...,...,...,...
162,TONIC_TMA20_R6C5,116,Lymphnode,non-responders
163,TONIC_TMA20_R6C6,116,Lymphnode,non-responders
164,TONIC_TMA20_R8C4,117,Skin,responders
165,TONIC_TMA20_R8C5,117,Skin,responders


### creating label column based on patients response to therapy. 
 0 = non-responder
 1 = responder

In [4]:

metadata['y'] = np.where(metadata['iRECIST_response']== 'responders', 1, 0)
metadata.head()

,fov,Patient_ID,Localization,iRECIST_response,y
0,TONIC_TMA2_R4C1,2,Other,non-responders,0
1,TONIC_TMA2_R4C2,2,Other,non-responders,0
2,TONIC_TMA2_R4C3,2,Other,non-responders,0
3,TONIC_TMA2_R9C6,5,Skin,responders,1
4,TONIC_TMA2_R10C1,5,Skin,responders,1


Tumor localization can be different for each patient based on where the cancer metastasized.

In [366]:
set(metadata['Localization'])

{'Breast', 'Liver', 'Lung', 'Lymphnode', 'Other', 'Skin', 'Thoracal'}

In [367]:
metadata.loc[metadata['Localization'] == 'Lymphnode']


,fov,Patient_ID,Localization,iRECIST_response,y
5,TONIC_TMA21_R7C1,4,Lymphnode,responders,1
6,TONIC_TMA21_R6C5,4,Lymphnode,responders,1
7,TONIC_TMA21_R6C6,4,Lymphnode,responders,1
8,TONIC_TMA10_R1C5,56,Lymphnode,responders,1
9,TONIC_TMA10_R5C2,57,Lymphnode,non-responders,0
...,...,...,...,...,...
159,TONIC_TMA20_R1C6,114,Lymphnode,non-responders,0
160,TONIC_TMA20_R7C1,116,Lymphnode,non-responders,0
161,TONIC_TMA20_R7C2,116,Lymphnode,non-responders,0
162,TONIC_TMA20_R6C5,116,Lymphnode,non-responders,0


In [5]:
aFileNames = metadata.loc[metadata['Localization'] == 'Lymphnode']['fov'].values.tolist()

aFileNamesLabels = metadata.loc[metadata['Localization'] == 'Lymphnode']['y'].values.tolist()


In [6]:
aFileNames_train, aFileNames_test, aFileNamesLabels_train, aFileNamesLabels_test = train_test_split(aFileNames, aFileNamesLabels, test_size=0.2, random_state=42)


66 training sample, 17 test, equal distribution of responders/non-responders

In [8]:
resnet_models = {18: torchvision.models.resnet18,
                 34: torchvision.models.resnet34,
                 50: torchvision.models.resnet50,
                 101: torchvision.models.resnet101,
                 152: torchvision.models.resnet152}


In [9]:
class Resnet_multichannel(nn.Module):
    def __init__(self, pretrained=True, encoder_depth=34, num_in_channels=4):
        super().__init__()
        
        if encoder_depth not in [18, 34, 50, 101, 152]:
            raise ValueError(f"Encoder depth {encoder_depth} specified does not match any existing Resnet models")
            
        model = resnet_models[encoder_depth](pretrained)
        
        print(model.conv1)
        
        ##For reference: layers to use (in order):
        # conv1, bn1, relu, maxpool, layer1, layer2, layer3, layer4, avgpool, fc
        
        # This increases the number of in channels for our network
        self.conv1 = self.increase_channels(model.conv1, num_in_channels)
        
        self.bn1 = model.bn1
        self.relu = model.relu
        self.maxpool = model.maxpool
        self.layer1 = model.layer1
        self.layer2 = model.layer2
        self.layer3 = model.layer3
        self.layer4 = model.layer4
        self.avgpool = model.avgpool
        
        self.fc = model.fc


        
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        
        
        return x
        
    def increase_channels(self, m, num_channels=None, copy_weights=0):
        """
        takes as input a Conv2d layer and returns the a Conv2d layer with `num_channels` input channels
        and all the previous weights copied into the new layer.
        """
        # number of input channels the new module should have
        new_in_channels = num_channels if num_channels is not None else m.in_channels + 1
        
        print(m.in_channels)
        print(new_in_channels)
       
        
        bias = False if m.bias is None else True
        
        # Creating new Conv2d layer
        new_m = nn.Conv2d(in_channels=new_in_channels, 
                          out_channels=m.out_channels, 
                          kernel_size=m.kernel_size, 
                          stride=m.stride, 
                          padding=m.padding,
                          bias=bias)
        
        # Copying the weights from the old to the new layer
        new_m.weight.data[:, :m.in_channels, :, :] = m.weight.clone()
        
        #Copying the weights of the `copy_weights` channel of the old layer to the extra channels of the new layer
        print(new_in_channels - m.in_channels)
        for i in range(new_in_channels - m.in_channels):
            channel = m.in_channels + i
            new_m.weight.data[:, channel:channel+1, :, :] = m.weight[:, copy_weights:copy_weights+1, : :].clone()
        new_m.weight = nn.Parameter(new_m.weight)

        return new_m
    


In [10]:
def get_arch(encoder_depth, num_in_channels):
    """
    Returns just an architecture which can then be called in the usual way.
    For example:
    resnet34_4_channel = get_arch(34, 4)
    model = resnet34_4_channel(True)
    """
    return partial(Resnet_multichannel, encoder_depth=encoder_depth, num_in_channels=num_in_channels)

In [11]:
markerNum = len(biomarker_toKeep)
markerNum

20

In [12]:
resnet50_mchannel = get_arch(50, markerNum)
#model = resnet50_mchannel(True) 

model = resnet50_mchannel(True) 

print("Total input channels : ", model.conv1.in_channels)

/home/bgsimon/.local/lib/python3.9/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
/home/bgsimon/.local/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
3
20
17
Total input channels :  20


In [13]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            #print(param)
            param.requires_grad = False
            
feature_extract = True
set_parameter_requires_grad(model, feature_extract)


In [14]:
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2) #2

In [394]:
print(summary(model))


Layer (type:depth-idx)                   Param #
Resnet_multichannel                      --
├─Conv2d: 1-1                            (62,720)
├─BatchNorm2d: 1-2                       (128)
├─ReLU: 1-3                              --
├─MaxPool2d: 1-4                         --
├─Sequential: 1-5                        --
│    └─Bottleneck: 2-1                   --
│    │    └─Conv2d: 3-1                  (4,096)
│    │    └─BatchNorm2d: 3-2             (128)
│    │    └─Conv2d: 3-3                  (36,864)
│    │    └─BatchNorm2d: 3-4             (128)
│    │    └─Conv2d: 3-5                  (16,384)
│    │    └─BatchNorm2d: 3-6             (512)
│    │    └─ReLU: 3-7                    --
│    │    └─Sequential: 3-8              (16,896)
│    └─Bottleneck: 2-2                   --
│    │    └─Conv2d: 3-9                  (16,384)
│    │    └─BatchNorm2d: 3-10            (128)
│    │    └─Conv2d: 3-11                 (36,864)
│    │    └─BatchNorm2d: 3-12            (128)
│    │    └─

Total params: 23,565,442

Trainable params: 4,098

Non-trainable params: 23,561,344

In [15]:
import torch.optim as optim
model = model.to(device)
params_to_update = model.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

Params to learn:
	 fc.weight
	 fc.bias


In [16]:
import time
import copy

def train_model(model, dataloaders, criterion, optimizer, num_epochs=25): #optimizer,
    since = time.time()

    val_acc_history = []
    val_loss_history = []
    labels_lst = []
    preds_lst = []
    weights = []
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            
            for inputs, labels in dataloaders[phase]:

                inputs = inputs.to(device)
                labels = labels.to(device)
                
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

                if epoch == num_epochs - 1:
                    labels_lst.append(labels)
                    preds_lst.append(preds)
                    

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            
            print("weights")
            print(model.state_dict()['fc.weight'][0])
            weights.append(model.state_dict()['fc.weight'][0])

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                print("better than best_acc")
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                
                val_acc_history.append(epoch_acc)
                val_loss_history.append(epoch_loss)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history, val_loss_history, time_elapsed, labels_lst, preds_lst, weights
                        
                        

In [17]:
experiment_name = 'pred_progress_v_0'
num_classes = 2
dir_experiment= '/oak/stanford/groups/ccurtis2/users/bgsimon/CS230_CourseProject/'
dir1 = dir_experiment+experiment_name
dir2 = dir_experiment+experiment_name+"/train/"
dir3 = dir_experiment+experiment_name+"/val/"

In [262]:
import os


dir1 = dir_experiment+experiment_name
dir2 = dir_experiment+experiment_name+"/train/"
dir3 = dir_experiment+experiment_name+"/val/"

if not os.path.exists(dir_experiment):
    
    os.mkdir(dir_experiment)

if not os.path.exists(dir1):
    os.mkdir(dir1)
if not os.path.exists(dir2):
    os.mkdir(dir2)
if not os.path.exists(dir3):
    os.mkdir(dir3)

    

uniqueLabels = [0,1]

for j in range(len(uniqueLabels)):
    n_dir = dir2+str(uniqueLabels[j])+'/'
    if not os.path.exists(n_dir):
        os.mkdir(n_dir)
for j in range(len(uniqueLabels)):
    n_dir = dir3+str(uniqueLabels[j])+'/'
    if not os.path.exists(n_dir):
        os.mkdir(n_dir)

    
for i in range(len(aFileNames_train)):
    file = dir2+str(aFileNamesLabels_train[i])+'/'+aFileNames_train[i]+'.tiff'
    open(file, 'a').close()

for i in range(len(aFileNames_test)):
    file = dir3+str(aFileNamesLabels_test[i])+'/'+aFileNames_test[i]+'.tiff'
    open(file, 'a').close()
    

In [18]:
from torchvision import datasets


RAW_Image_Source_Folder = '/oak/stanford/groups/ccurtis2/users/bgsimon/CS230_CourseProject/pred_progress_v_0_images/'

def my_tiff_loader(filename):    
    
    head_tail = os.path.split(filename)
    filenameWithTiff = head_tail[-1]
    
    
    fileNameToBeUsed = filenameWithTiff[0:-(len('.tiff'))]    
        
    folder_path = RAW_Image_Source_Folder+fileNameToBeUsed+'/'
    
    cdir = !ls $folder_path

    for n in cdir:
        if n[-5:] == '.tiff':
            newn = n.replace(".tiff", ".tif" )
            os.rename(folder_path+n, folder_path+newn)

    i=0
    cdir = !ls $folder_path
    for my_tif in cdir:

        biomarkerName = my_tif[0:-(len('.tif'))]
        if biomarkerName in biomarker_toKeep:
        
            image_dir = folder_path+my_tif

            if i==0:
                im = Image.open(image_dir)
                im = im.resize([224,224])
                image = to_tensor(im)
            else:
                im = Image.open(image_dir) 
                im = im.resize([224,224])
                next_channel = to_tensor(im)
                image = torch.cat((image,next_channel),0)
            i += 1

    return image
    

In [19]:

image_datasets = {x: datasets.ImageFolder(os.path.join(dir1, x),loader=my_tiff_loader) for x in ['train','val']} # ,val 

dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=1, shuffle=True, num_workers=0) for x in ['train','val']} #val

dataloaders_dict.values()



dict_values([<torch.utils.data.dataloader.DataLoader object at 0x2b6ed8dad6d0>, <torch.utils.data.dataloader.DataLoader object at 0x2b6ed8dad8e0>])

In [20]:
image_datasets

{'train': Dataset ImageFolder
     Number of datapoints: 66
     Root location: /oak/stanford/groups/ccurtis2/users/bgsimon/CS230_CourseProject/pred_progress_v_0/train,
 'val': Dataset ImageFolder
     Number of datapoints: 17
     Root location: /oak/stanford/groups/ccurtis2/users/bgsimon/CS230_CourseProject/pred_progress_v_0/val}

In [21]:
image_datasets.values()

dict_values([Dataset ImageFolder
    Number of datapoints: 66
    Root location: /oak/stanford/groups/ccurtis2/users/bgsimon/CS230_CourseProject/pred_progress_v_0/train, Dataset ImageFolder
    Number of datapoints: 17
    Root location: /oak/stanford/groups/ccurtis2/users/bgsimon/CS230_CourseProject/pred_progress_v_0/val])

In [22]:
model, val_acc_history, val_loss_history, time_elapsed, labels_lst, preds_lst, weights = train_model(model,
                                     dataloaders = dataloaders_dict, 
                                     criterion = torch.nn.CrossEntropyLoss(),
                                     optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9),
                                     num_epochs=50)
#optimizer = torch.optim.Adam,

Epoch 0/49
----------
train Loss: 0.9111 Acc: 0.6364
weights
tensor([-0.0131,  0.0204, -0.0174,  ..., -0.0058,  0.0029,  0.0112])
val Loss: 1.5050 Acc: 0.7059
weights
tensor([-0.0131,  0.0204, -0.0174,  ..., -0.0058,  0.0029,  0.0112])
better than best_acc

Epoch 1/49
----------
train Loss: 0.9977 Acc: 0.7121
weights
tensor([-1.5105e-02,  1.9141e-02, -1.8671e-02,  ..., -8.5395e-03,
        -4.6901e-05,  9.0248e-03])
val Loss: 0.6728 Acc: 0.5294
weights
tensor([-1.5105e-02,  1.9141e-02, -1.8671e-02,  ..., -8.5395e-03,
        -4.6901e-05,  9.0248e-03])

Epoch 2/49
----------
train Loss: 1.3216 Acc: 0.5606
weights
tensor([-0.0151,  0.0202, -0.0177,  ..., -0.0090,  0.0014,  0.0095])
val Loss: 0.6972 Acc: 0.7059
weights
tensor([-0.0151,  0.0202, -0.0177,  ..., -0.0090,  0.0014,  0.0095])

Epoch 3/49
----------
train Loss: 1.0301 Acc: 0.6364
weights
tensor([-0.0171,  0.0188, -0.0192,  ..., -0.0110, -0.0009,  0.0080])
val Loss: 1.8819 Acc: 0.2941
weights
tensor([-0.0171,  0.0188, -0.0192,  .

In [23]:
val_loss_history

[1.5050362968878985,
 0.6728084508110496,
 0.6971796610776115,
 1.881883674465558,
 1.1812916196444456,
 1.5440019884785323,
 0.6636449725312346,
 0.9195905088940087,
 1.2457574425703462,
 1.6937685438189125,
 0.8420120554373545,
 1.442481076474959,
 1.7067869007587433,
 1.6844997112630689,
 1.5831641084993915,
 0.8306915802771554,
 0.8223116935833412,
 0.7663666313404546,
 1.0365187055705225,
 1.0276007433147991,
 0.818581648170948,
 1.4575308415223844,
 1.3635745785380577,
 1.001259610933416,
 1.4968719283745433,
 1.0295357582542826,
 2.8948114066697266,
 1.2733678668737411,
 2.204823799414311,
 1.6243335346790606,
 1.8819410593973505,
 1.124245326637345,
 2.5292590371770047,
 0.9338270201731254,
 1.1195662628366227,
 0.9163765801007256,
 1.729351720534717,
 1.3382510571973398,
 1.2315647352267713,
 1.4050415633520221,
 2.349367411040208,
 3.200935031954386,
 2.066686276495388,
 1.5063999934645835,
 1.1810385312282425,
 1.577321155998267,
 1.5879771693075515,
 1.8227917153135957,
 1.

In [24]:
val_acc_history

[tensor(0.7059, dtype=torch.float64),
 tensor(0.5294, dtype=torch.float64),
 tensor(0.7059, dtype=torch.float64),
 tensor(0.2941, dtype=torch.float64),
 tensor(0.2941, dtype=torch.float64),
 tensor(0.7059, dtype=torch.float64),
 tensor(0.6471, dtype=torch.float64),
 tensor(0.7059, dtype=torch.float64),
 tensor(0.7059, dtype=torch.float64),
 tensor(0.7059, dtype=torch.float64),
 tensor(0.7059, dtype=torch.float64),
 tensor(0.7059, dtype=torch.float64),
 tensor(0.2941, dtype=torch.float64),
 tensor(0.7059, dtype=torch.float64),
 tensor(0.7059, dtype=torch.float64),
 tensor(0.6471, dtype=torch.float64),
 tensor(0.7059, dtype=torch.float64),
 tensor(0.6471, dtype=torch.float64),
 tensor(0.7059, dtype=torch.float64),
 tensor(0.4118, dtype=torch.float64),
 tensor(0.6471, dtype=torch.float64),
 tensor(0.7059, dtype=torch.float64),
 tensor(0.7059, dtype=torch.float64),
 tensor(0.4118, dtype=torch.float64),
 tensor(0.7059, dtype=torch.float64),
 tensor(0.6471, dtype=torch.float64),
 tensor(0.70

In [ ]:
#df_loss_acc = pd.DataFrame(history.history)
df_loss= df_loss_acc[['loss','val_loss']]
df_loss.rename(columns={'loss':'train','val_loss':'validation'},inplace=True)
df_acc= df_loss_acc[['binary_accuracy','val_binary_accuracy']]
df_acc.rename(columns={'binary_accuracy':'train','val_binary_accuracy':'validation'},inplace=True)
df_loss.plot(title='Model loss',figsize=(12,8)).set(xlabel='Epoch',ylabel='Loss')
df_acc.plot(title='Model Accuracy',figsize=(12,8)).set(xlabel='Epoch',ylabel='Accuracy')


In [39]:
print(str(int(time_elapsed // 60)) + " minutes, " + str(int(time_elapsed % 60)) + " seconds")

205 minutes, 25 seconds


False Positive: A non-responder who is predicted to be a responder

False Negative: A responder who is predicted to be a non-responder

True Positive: A responder who is predicted to be a responder

True Negative: A non-responder who is predicted to be a non-responder

In [25]:
fp = 0
fn = 0
tp = 0
tn = 0
for i in range(len(labels_lst)):
    if labels_lst[i] == 0 and preds_lst[i] == 1:
        fp += 1
    if labels_lst[i] == 1 and preds_lst[i] == 0:
        fn += 1
    if labels_lst[i] == 1 and preds_lst[i] == 1:
        tp += 1
    if labels_lst[i] == 0 and preds_lst[i] == 0:
        tn += 1
    
fp = fp / len(labels_lst)
fn = fn / len(labels_lst)
tp = tp / len(labels_lst)
tn = tn / len(labels_lst)

print("False positive rate: " + str(fp))
print("False negative rate: " + str(fn))
print("True positive rate: " + str(tp))
print("True negative rate: " + str(tn))

False positive rate: 0.13253012048192772
False negative rate: 0.1686746987951807
True positive rate: 0.08433734939759036
True negative rate: 0.6144578313253012
